# K-Nearest-Neighbors (KNN)


## Importing Libraries

In [21]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

#Importing the data
## About the data

<img src='https://images.squarespace-cdn.com/content/v1/5c36736d9d5abb8b44c6b7f7/1586468701075-G14EXPZE2VNMJFKDCQ0T/rook+and+king+logo.jpg?format=500w' align="center"><br>
  Chess End-Game -- King+Rook versus King+Pawn on a7
  (usually abbreviated KRKPA7).  The pawn on a7 means it is one square
  away from queening.  It is the King+Rook's side (white) to move.

  

In [22]:
king_data=pd.read_excel('King-Rook vs. King-Pawn.xlsx')

In [23]:
king_data.head(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
5,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,t,f,f,f,t,t,n,won
6,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
7,f,f,f,f,t,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
8,f,f,f,f,f,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,t,f,f,t,t,n,won
9,f,f,f,f,f,f,t,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won


In [24]:
king_data.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21', 'Column22', 'Column23', 'Column24',
       'Column25', 'Column26', 'Column27', 'Column28', 'Column29', 'Column30',
       'Column31', 'Column32', 'Column33', 'Column34', 'Column35', 'Column36',
       'Column37'],
      dtype='object')

In [25]:
object_col=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21', 'Column22', 'Column23', 'Column24',
       'Column25', 'Column26', 'Column27', 'Column28', 'Column29', 'Column30',
       'Column31', 'Column32', 'Column33', 'Column34', 'Column35', 'Column36'
       ]

Droping NaN values

In [26]:
king_data.dropna(axis=0)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3191,t,f,f,f,f,f,t,f,f,f,f,t,l,f,b,f,f,t,f,f,f,f,t,t,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3192,t,f,f,f,f,f,t,f,f,f,f,t,l,f,b,f,f,t,f,f,f,f,t,f,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3193,t,f,f,f,f,f,t,f,f,f,f,t,l,f,n,f,f,t,f,f,f,f,t,f,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3194,t,f,t,f,f,f,t,f,f,f,f,t,l,t,b,t,f,t,f,f,f,f,f,t,f,t,f,f,t,f,f,t,f,f,f,n,nowin


In [27]:
print(f'{king_data.Column37.unique()}')

['won' 'nowin']


In [28]:
king_data['Column37']=king_data['Column37'].apply({'won':2,'nowin':1}.get)

In [29]:
king_data.head(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,2
5,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,t,f,f,f,t,t,n,2
6,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,t,f,t,f,f,f,f,f,f,f,t,t,n,2
7,f,f,f,f,t,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
8,f,f,f,f,f,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,t,f,f,t,t,n,2
9,f,f,f,f,f,f,t,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2


## Spliting the data.

After checking through various splits, the best promising split is chosen **85 : 15 train - test split.**

P.S. Using the splits 90-10, 80-20, 75-25 splits were also sharing better results than the latest version but the selected split shared the best among all.

In [30]:
X=king_data.copy()
y=X.pop('Column37')

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,train_size=0.3)


In [31]:
y_test.head(10)

2925    1
306     2
1345    1
914     1
240     2
1856    2
2835    1
2441    1
2210    2
1046    1
Name: Column37, dtype: int64

## Training and predicting the model using KNN algo.



In [32]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

one_hot_train=pd.DataFrame(OH_encoder.fit_transform(X_train[object_col]))
one_hot_test=pd.DataFrame(OH_encoder.transform(X_test[object_col]))
one_hot_train.index = X_train.index
one_hot_test.index = X_test.index

In [33]:
num_X_train = X_train.drop(object_col, axis=1)
num_X_valid = X_test.drop(object_col, axis=1)

In [34]:
OH_X_train = pd.concat([num_X_train, one_hot_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, one_hot_test], axis=1)

In [35]:
y_train.unique()

array([2, 1])

In [36]:
from sklearn.neighbors import KNeighborsClassifier as knn

model=knn(n_neighbors=1,
          leaf_size=1,
          algorithm='brute',
          weights='distance',
          p=1
          )

model.fit(OH_X_train,y_train)

KNeighborsClassifier(algorithm='brute', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=1,
                     weights='distance')

In [37]:
pred=model.predict(OH_X_valid)

## Checking model accuracy.


In [38]:
from sklearn.metrics import accuracy_score

x=accuracy_score(y_test,pred)
print(f'the accuracy score is {x}')

the accuracy score is 0.88125


## Using confusion matric

In [39]:
from sklearn.metrics import confusion_matrix

cm= confusion_matrix(y_test,pred)

In [40]:
print(cm)

[[200  32]
 [ 25 223]]
